# Natural Language Processing1

텍스트에서 의미있는 정보를 분석, 추출하고 이해하는 일련의 기술집합이다.

## Tokenize 

In [1]:
from nltk.corpus import gutenberg

In [2]:
gutenberg.fileids() #nltk의 corpus에 있는 파일들

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
corpus = gutenberg.open('austen-emma.txt').read() # corpus의 파일을 읽어온다.

In [4]:
from nltk import regexp_tokenize, Text 
# regular expression을 이용해서 해당하는 패턴을 만족하는 text찾을 것

In [5]:
pattern = r"([a-zA-Z0-9]+)+" #정규식으로 해당 패턴을 만들어 준다.
# 영어, 숫자 조합으로 1개 이상의 글자를 지니는 그룹을 1개 이상 가지는 패턴

In [6]:
tokens = regexp_tokenize(corpus, pattern) #해당 corpus를 pattern에 맞게 tokenize해줌

In [7]:
len(tokens)

161980

In [8]:
en = Text(tokens)

In [9]:
type(en) # token들을 Text 객체로 만들어줌.

nltk.text.Text

In [10]:
from konlpy.corpus import kolaw

In [11]:
corpus = kolaw.open("constitution.txt").read() # kolaw에 있는 corpus를 들고온다.

In [12]:
from konlpy.tag import Kkma # 꼬꼬마 형태소 분석기를 들고옴.

In [13]:
kkma = Kkma() # 꼬고마 형태소 객체 생성

In [14]:
tokens = kkma.morphs(corpus) # 꼬꼬마 형태소 분석기로 corpus를 토큰화해줌.

In [15]:
tokens[:5]

['대한민국', '헌법', '유구', '하', 'ㄴ']

In [16]:
ko = Text(tokens) # 한국어 토큰들에 대해 Text 객체를 만들어줌

## collocation :
#### 2개 이상의 단어가 함께 의미를 전달하기 위해 사용되는 단어들의 덩어리  

### 임의로 자신이 만든 rule에 대해 parsing tree 구축

In [17]:
sentence = "The little yellow dog barked at the Persian cat"

In [18]:
tokens = sentence.split() # 어절 단위로 (Whitespace 단위로) 토큰화해준다.

In [19]:
import nltk

In [20]:
nltk.__version__

'3.2.4'

In [21]:
enTags = nltk.pos_tag(tokens) # 토큰에 대해서 품사 tagging해준다.
# 튜플 리스트로 리턴해준다.

In [22]:
enTags

[('The', 'DT'),
 ('little', 'JJ'),
 ('yellow', 'JJ'),
 ('dog', 'NN'),
 ('barked', 'VBD'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('Persian', 'JJ'),
 ('cat', 'NN')]

In [23]:
sentence = "내 친구는 잠을 많이 잔다."

In [24]:
koTags = kkma.pos(sentence) # 꼬꼬마 형태소 분석기로 토큰화 및 품사 태깅 해준다.

In [25]:
koTags

[('내', 'NP'),
 ('친구', 'NNG'),
 ('는', 'JX'),
 ('잠', 'NNG'),
 ('을', 'JKO'),
 ('많이', 'MAG'),
 ('자', 'VV'),
 ('ㄴ다', 'EFN'),
 ('.', 'SF')]

In [26]:
enParser = nltk.RegexpParser("NP: {<DT>?<JJ>*<NN.*>*}") 
# 정규식에 맞게 품사 태깅 해줄 수 있게 해줌.
# 관사<DT>가 있을수도 있고, 형용사<JJ> 0개 이상, 
# ( <NN> 뒤에 무엇이 0개이상 붙는 것 )이 0개 이상 인것..

In [27]:
tree = enParser.parse(enTags) 
#해당 Parser의 grammar에 따라서 parse tree 생성해줌.

In [28]:
tree.draw() # 해당 parse tree를 그려줌. 새창으로 띄워준다.

In [29]:
tree.pprint() # 이렇게 출력해주기도 한다.

(S
  (NP The/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT Persian/JJ cat/NN))


In [30]:
koParser = nltk.RegexpParser("""
NP: {<N.*>*<N.*>?}
OP: {<N.*>*<JKO>*}
VP: {<M.*>*<V.*>*<E.*>*}
""") # 해당 정규식 grammar에 맞게 parser를 만들어준다.
#명사구(NP)는 (N으로 시작하고, N뒤에 0개 이상이 붙는것)이 0개 이상이고, 또하나가 더 올 수 있는 것 
#동사구(VP)는 (M으로 시작하는 것이 0개 이상, V로 시작하는것이 0개이상, E로 시작하는 것이 0개 이상)


In [31]:
tree = koParser.parse(koTags) # 위의 parser에 맞게 parse tree 만들어줌

In [32]:
tree.draw() # parse tree를 만들어준다.

In [33]:
tree.pprint()

(S
  (OP (NP 내/NP 친구/NNG))
  는/JX
  (OP (NP 잠/NNG) 을/JKO)
  (VP 많이/MAG 자/VV ㄴ다/EFN)
  ./SF)


In [34]:
for subtree in tree.subtrees(): #서브트리 for문 돌린다.
    if subtree.label() == "VP": # 만약 tag가 VP일때
        print(subtree,type(subtree)) #subtree와 타입을 출력
        print(list(subtree)) #subtree의 토큰 된것들 리스트화 해서 출력
        print(" ".join(e[0] for e in list(subtree))) #토큰화된것들 묶어서 출력

(VP 많이/MAG 자/VV ㄴ다/EFN) <class 'nltk.tree.Tree'>
[('많이', 'MAG'), ('자', 'VV'), ('ㄴ다', 'EFN')]
많이 자 ㄴ다


### collocation 

In [35]:
en.collocations() #단어쌍이 많은 것들 출력해준다. (연어)

Mrs Weston; Frank Churchill; Miss Woodhouse; Mrs Elton; Miss Bates;
Jane Fairfax; Miss Fairfax; every thing; young man; every body; great
deal; Mrs Goddard; dare say; Maple Grove; John Knightley; Miss Taylor;
Miss Smith; Robert Martin; Colonel Campbell; Box Hill


In [36]:
ko.collocations(window_size=3) #단어의 위치 기반으로 단어쌍이 많은 것들 출력해준다.
#연속된 단어 중에 window 크기를 3으로 해서 많이 나오는 단어쌍을 출력해준다.

정치적 중립성; 재판소 재판관; 대법원장 대법관; 대통령 국무총리; 그러하 아니하


PMI : 2개의 독립적인 이산 확률을 통해서 2가지의 관계성을 확인할 수 있도록 해준다.

<br>감성분석, opinion mining을 하는데 쓰인다.

### Bigram

확률론적 언어 모형(Probabilistic Language Model)은  m 개의 단어  $w1,w2,…,wm$  열(word sequence)이 주어졌을 때 문장으로써 성립될 확률  $P(w1,w2,…,wm)$  을 출력함으로써 이 단어 열이 실제로 현실에서 사용될 수 있는 문장(sentence)인지를 판별하는 모형이다.

만약 단어의 활용이 바로 전 단어에만 의존한다면 단어 열의 확률은 다음과 같다. 이러한 모형을 Bigram 모형 또는 마코프 모형(Markov Model)이라고 한다.

$$
P(w1,w2,…,wm)=P(w1) ∏ i=2mP(wi|wi−1) 
$$

[확률론적 언어 모델 설명 링크](https://datascienceschool.net/view-notebook/a0c848e1e2d343d685e6077c35c4203b/)

In [37]:
from nltk import collocations
#연어를 활용해서 통계적으로 관련성이 있는 것들을 알려고 한다.

In [38]:
measure = collocations.BigramAssocMeasures()
# bigram 객체 생성

In [39]:
taggedWords = kkma.pos(corpus) # corpus에 대해서 tokenizing 및 tagging해준다.

In [40]:
taggedWords

[('대한민국', 'NNG'),
 ('헌법', 'NNG'),
 ('유구', 'NNG'),
 ('하', 'XSV'),
 ('ㄴ', 'ETD'),
 ('역사', 'NNG'),
 ('와', 'JC'),
 ('전통', 'NNG'),
 ('에', 'JKM'),
 ('빛나', 'VV'),
 ('는', 'ETD'),
 ('우리', 'NNM'),
 ('대하', 'VV'),
 ('ㄴ', 'ETD'),
 ('국민', 'NNG'),
 ('은', 'JX'),
 ('3', 'NR'),
 ('·', 'SP'),
 ('1', 'NR'),
 ('운동', 'NNG'),
 ('으로', 'JKM'),
 ('건립', 'NNG'),
 ('되', 'XSV'),
 ('ㄴ', 'ETD'),
 ('대한민국', 'NNG'),
 ('임시', 'NNG'),
 ('정부', 'NNG'),
 ('의', 'JKG'),
 ('법통', 'NNG'),
 ('과', 'JC'),
 ('불의', 'NNG'),
 ('에', 'JKM'),
 ('항거', 'NNG'),
 ('하', 'XSV'),
 ('ㄴ', 'ETD'),
 ('4', 'NR'),
 ('·', 'SP'),
 ('19', 'NR'),
 ('민주', 'NNG'),
 ('이념', 'NNG'),
 ('을', 'JKO'),
 ('계승', 'NNG'),
 ('하', 'XSV'),
 ('고', 'ECE'),
 (',', 'SP'),
 ('조국', 'NNG'),
 ('의', 'JKG'),
 ('민주', 'NNG'),
 ('개혁', 'NNG'),
 ('과', 'JKM'),
 ('평화적', 'NNG'),
 ('통일', 'NNG'),
 ('의', 'JKG'),
 ('사명', 'NNG'),
 ('에', 'JKM'),
 ('입각', 'NNG'),
 ('하', 'XSV'),
 ('여', 'ECS'),
 ('정의', 'NNG'),
 ('·', 'SP'),
 ('인도', 'NNG'),
 ('와', 'JC'),
 ('동포애', 'NNG'),
 ('로써', 'JKM'),
 ('민족', 'NNG'),

In [41]:
finder = collocations.BigramCollocationFinder.from_words(taggedWords)
#이미 형태소 분석이 되어 있으므로 단어에서 Bigram을 갖고 온다.

In [42]:
for row in finder.nbest(measure.pmi, 5):
    print(row)
#pmi 척도로 상위 5개의 ,bigram 쌍을 출력할 것이다.

(('가부', 'NNG'), ('동수', 'NNG'))
(('강제', 'NNG'), ('노역', 'NNG'))
(('경자', 'NNG'), ('유전', 'NNG'))
(('고', 'ECS'), ('채취', 'NNG'))
(('공무', 'NNG'), ('담임', 'NNG'))


In [43]:
tags = [t for (w, t) in taggedWords] # tag들을 list로 묶어준다.

In [44]:
tags

['NNG',
 'NNG',
 'NNG',
 'XSV',
 'ETD',
 'NNG',
 'JC',
 'NNG',
 'JKM',
 'VV',
 'ETD',
 'NNM',
 'VV',
 'ETD',
 'NNG',
 'JX',
 'NR',
 'SP',
 'NR',
 'NNG',
 'JKM',
 'NNG',
 'XSV',
 'ETD',
 'NNG',
 'NNG',
 'NNG',
 'JKG',
 'NNG',
 'JC',
 'NNG',
 'JKM',
 'NNG',
 'XSV',
 'ETD',
 'NR',
 'SP',
 'NR',
 'NNG',
 'NNG',
 'JKO',
 'NNG',
 'XSV',
 'ECE',
 'SP',
 'NNG',
 'JKG',
 'NNG',
 'NNG',
 'JKM',
 'NNG',
 'NNG',
 'JKG',
 'NNG',
 'JKM',
 'NNG',
 'XSV',
 'ECS',
 'NNG',
 'SP',
 'NNG',
 'JC',
 'NNG',
 'JKM',
 'NNG',
 'JKG',
 'NNG',
 'JKO',
 'MAG',
 'VV',
 'ECE',
 'SP',
 'MDT',
 'NNG',
 'NNG',
 'JC',
 'NNG',
 'JKO',
 'NNG',
 'XSV',
 'ECE',
 'SP',
 'NNG',
 'JC',
 'NNG',
 'JKO',
 'NNG',
 'JKM',
 'NNG',
 'NNG',
 'NNG',
 'NNG',
 'JKO',
 'MAG',
 'MAG',
 'MAG',
 'NNG',
 'SP',
 'NNG',
 'SP',
 'NNG',
 'SP',
 'NNG',
 'JKG',
 'MDT',
 'NNG',
 'JKM',
 'VV',
 'ECD',
 'NNG',
 'JKG',
 'NNG',
 'JKO',
 'NNG',
 'NNG',
 'VV',
 'ECE',
 'SP',
 'NNG',
 'JKO',
 'NNG',
 'NNG',
 'NNG',
 'XSV',
 'ECD',
 'VV',
 'ECE',
 'SP',
 'N

In [45]:
words = [w for (w,t) in taggedWords] # 단어들을 list로 묶어준다.

In [46]:
words 

['대한민국',
 '헌법',
 '유구',
 '하',
 'ㄴ',
 '역사',
 '와',
 '전통',
 '에',
 '빛나',
 '는',
 '우리',
 '대하',
 'ㄴ',
 '국민',
 '은',
 '3',
 '·',
 '1',
 '운동',
 '으로',
 '건립',
 '되',
 'ㄴ',
 '대한민국',
 '임시',
 '정부',
 '의',
 '법통',
 '과',
 '불의',
 '에',
 '항거',
 '하',
 'ㄴ',
 '4',
 '·',
 '19',
 '민주',
 '이념',
 '을',
 '계승',
 '하',
 '고',
 ',',
 '조국',
 '의',
 '민주',
 '개혁',
 '과',
 '평화적',
 '통일',
 '의',
 '사명',
 '에',
 '입각',
 '하',
 '여',
 '정의',
 '·',
 '인도',
 '와',
 '동포애',
 '로써',
 '민족',
 '의',
 '단결',
 '을',
 '공고히',
 '하',
 '고',
 ',',
 '모든',
 '사회적',
 '폐습',
 '과',
 '불의',
 '를',
 '타파',
 '하',
 '며',
 ',',
 '자율',
 '과',
 '조화',
 '를',
 '바탕',
 '으로',
 '자유',
 '민주적',
 '기본',
 '질서',
 '를',
 '더욱',
 '확고히',
 '하여',
 '정치',
 '·',
 '경제',
 '·',
 '사회',
 '·',
 '문화',
 '의',
 '모든',
 '영역',
 '에',
 '있',
 '어서',
 '각인',
 '의',
 '기회',
 '를',
 '균등',
 '히',
 '하',
 '고',
 ',',
 '능력',
 '을',
 '최고',
 '도로',
 '발휘',
 '하',
 '게',
 '하',
 '며',
 ',',
 '자유',
 '와',
 '권리',
 '에',
 '따르',
 '는',
 '책임',
 '과',
 '의무',
 '를',
 '완수',
 '하',
 '게',
 '하여',
 ',',
 '안',
 '으로',
 '는',
 '국민',
 '생활',
 '의',
 '균등',
 '하',
 'ㄴ',
 

In [47]:
finder = collocations.BigramCollocationFinder.from_words(tags)

for row in finder.nbest(measure.pmi, 5): #pmi의 척도로 상위 5개의 bigram tag를 갖고옴
    print(row) 
#이를 통해서 각 태그에 대해서 어떠한 태그 뒤에는 어떤 태그가 많이 나온다는 것을 알 수 있다.


('XR', 'XSA')
('JKC', 'VCN')
('EPT', 'EPT')
('VCN', 'ECD')
('ECD', 'VX')


In [48]:
finder = collocations.BigramCollocationFinder.from_words(words)

for row in finder.nbest(measure.pmi, 5): #pmi의 척도로 상위 5개의 bigram 단어를 갖고옴
    print(row) 
#이를 통해서 각 단어에 대해서 어떠한 단어 뒤에는 어떤 단어가 많이 나온다는 것을 알 수 있다.
# 많이 나오는 단어 쌍을 알 수 있다.

('가부', '동수')
('강제', '노역')
('경자', '유전')
('공무', '담임')
('공중', '도덕')


### Word Cloud 생성

In [49]:
ko.vocab() 
# 해당 token들에 대해서 빈도수를 value로 가지는 dict타입 단어장 만들어준다.

FreqDist({'대한민국': 11,
          '헌법': 69,
          '유구': 1,
          '하': 457,
          'ㄴ': 234,
          '역사': 1,
          '와': 42,
          '전통': 1,
          '에': 328,
          '빛나': 1,
          '는': 281,
          '우리': 3,
          '대하': 19,
          '국민': 69,
          '은': 195,
          '3': 24,
          '·': 145,
          '1': 28,
          '운동': 3,
          '으로': 57,
          '건립': 1,
          '되': 113,
          '임시': 3,
          '정부': 23,
          '의': 532,
          '법통': 1,
          '과': 82,
          '불의': 2,
          '항거': 1,
          '4': 14,
          '19': 2,
          '민주': 6,
          '이념': 1,
          '을': 232,
          '계승': 2,
          '고': 39,
          ',': 101,
          '조국': 3,
          '개혁': 1,
          '평화적': 4,
          '통일': 9,
          '사명': 2,
          '입각': 2,
          '여': 149,
          '정의': 2,
          '인도': 1,
          '동포애': 1,
          '로써': 5,
          '민족': 3,
          '단결': 1,
          '공고히': 1,
         

In [50]:
with open("cloud.csv", "w", encoding="utf-8") as f:
    f.write("word, freq\n") 
    for k,v in ko.vocab().items():
        f.write("{0}, {1}\n".format(k,v))

In [51]:
!pip install simplejson
!pip install pygame
!pip install pytagcloud #단어 빈도에 따라 크기, 색을 다르게 하여 이미지로 저장해줌.

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Invalid requirement: '#단어'
Traceback (most recent call last):
  File "d:\anaconda3\lib\site-packages\pip\_vendor\packaging\requirements.py", line 93, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "d:\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1632, in parseString
    raise exc
  File "d:\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1622, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "d:\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "d:\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 3395, in parseImpl
    loc, exprtokens = e._parse( instring, loc, doActions )
  File "d:\anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _pa

In [52]:
wordList = [k for (k,v) in kkma.pos(corpus) if v.startswith("NN")]  # 형태소 분석한 결과 중에 태그가 NN으로 시작한다면 받아옴

In [53]:
wordList

['대한민국',
 '헌법',
 '유구',
 '역사',
 '전통',
 '우리',
 '국민',
 '운동',
 '건립',
 '대한민국',
 '임시',
 '정부',
 '법통',
 '불의',
 '항거',
 '민주',
 '이념',
 '계승',
 '조국',
 '민주',
 '개혁',
 '평화적',
 '통일',
 '사명',
 '입각',
 '정의',
 '인도',
 '동포애',
 '민족',
 '단결',
 '사회적',
 '폐습',
 '불의',
 '타파',
 '자율',
 '조화',
 '바탕',
 '자유',
 '민주적',
 '기본',
 '질서',
 '정치',
 '경제',
 '사회',
 '문화',
 '영역',
 '각인',
 '기회',
 '균등',
 '히',
 '능력',
 '최고',
 '도로',
 '발휘',
 '자유',
 '권리',
 '책임',
 '의무',
 '완수',
 '안',
 '국민',
 '생활',
 '균등',
 '향상',
 '밖',
 '항구적',
 '세계',
 '평화',
 '인류',
 '공영',
 '이바지',
 '자손',
 '안전',
 '자유',
 '행복',
 '확보',
 '것',
 '다짐',
 '년',
 '월',
 '일',
 '제정',
 '차',
 '개정',
 '헌법',
 '이제',
 '국회',
 '의결',
 '국민',
 '투표',
 '개정',
 '장',
 '강',
 '제',
 '대한민국',
 '민주',
 '공화국',
 '대한민국',
 '주권',
 '국민',
 '권력',
 '국민',
 '대한민국',
 '국민',
 '요건',
 '법률',
 '국가',
 '법률',
 '바',
 '재외',
 '국민',
 '보호',
 '의무',
 '조',
 '대한민국',
 '영토',
 '한반도',
 '부속',
 '조',
 '대한민국',
 '통일',
 '지향',
 '자유',
 '민주적',
 '기본',
 '질서',
 '입각',
 '평화적',
 '통일',
 '정책',
 '수립',
 '추진',
 '대한민국',
 '국제',
 '평화',
 '유지',
 '노력',
 '침략',
 '전쟁',
 '부인',
 '국',
 '

In [54]:
from collections import Counter
# collections.Counter는 hashable 객체들을 카운팅해주는 dict타입 서브클래스

In [55]:
wordCount = Counter(wordList) # 위의 단어 리스트를 카운팅해주는 Counter객체 생성

In [56]:
wordCount = wordCount.most_common(40) #높은 빈도수를 가지는 최상위 40개의 단어들을 갖고 온다.

In [57]:
wordCount

[('법률', 121),
 ('수', 88),
 ('대통령', 84),
 ('국가', 73),
 ('헌법', 69),
 ('국민', 69),
 ('조', 58),
 ('국회', 55),
 ('때', 55),
 ('회의', 42),
 ('바', 37),
 ('필요', 31),
 ('위원', 31),
 ('국무', 30),
 ('기타', 26),
 ('선거', 26),
 ('보장', 24),
 ('정부', 23),
 ('사항', 23),
 ('의원', 23),
 ('항', 22),
 ('자유', 21),
 ('권리', 21),
 ('국', 21),
 ('모든', 21),
 ('의무', 20),
 ('제', 20),
 ('일', 19),
 ('직무', 19),
 ('임명', 19),
 ('경제', 18),
 ('조직', 18),
 ('이상', 18),
 ('국회의원', 18),
 ('임기', 18),
 ('공무원', 17),
 ('경우', 17),
 ('법원', 17),
 ('의결', 16),
 ('단체', 16)]

In [58]:
import pytagcloud # 태그 클라우드를 만들 수 있게 해주는 라이브러리

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [59]:
tagList = pytagcloud.make_tags(wordCount, maxsize=40) #태그 리스트를 만든다. size, 색을 알아서 지정해줌.

In [60]:
pytagcloud.create_tag_image(tagList, "cloud.jpg", fontname="NanumGothic") # 위에서 만들어진 태그 리스트에 대해서 이미지로 저장해준다.

#### EX) 친구들과의 단톡방 데이터로 어떤 단어가 자주 나왔는지 WordCloud 만들기

In [61]:
text = open("D:\Study\산학연계\챗봇\데이터셋\가공처리된것\데이터셋1.txt", encoding='utf-8').read() 

In [62]:
text = text.replace('\n','')[3:] # \n들 다 없애줌

In [80]:
import re

In [104]:
text_edit = ""
for i in re.findall(r'[가-힣\s]',text):
    text_edit += i

In [105]:
text_edit

'민방위  교육  스물셋에 민방위라니  넹  개심심  아 드디어    누난 안 가잔아여  노잼일거같아서  상당히  아  아 왜    셤  넌 대구잖아  기간  어  당연하지  언제  아 배고파  까지임  다음주월요일부터  금요일까지  나는그래  다들  그렇겟고만  넌  대구에서뭐하고사냐  개잉여네    코딩이나해    무슨  직장인들이 제일 부러워하는게  이런 소중한시간에  씨언어 한장을  더공부해라  백수임  감기  걸렷다  그럼  누워  서  쉬어야지  쉬어  와 부럽다  아직 복학이 달이나 남았어  와  개부럽다  아 년 더 하고싶다  넌언제복학  학기죠  읭  반학기더한거야  아닌가 작년 여름에했나  보네    아 그때 유럽갔지  크  기억력  늙엇네  그리고  오늘 노잼민방위하면서  신세한탄중이지  무슨  안보교육  글너거함  민방위라는거하면  넹  노잼이네  핵노잼  언제끝남  근데대체  왜 이시간까지그럼  끝나가네요    아살려줘  진짜  다던지고싶다  토탁토닥  담주가중간고사임  이 응  껄껄담주에휴가자들많던데  재밋겟군  유나누나  놋북살때  인터넷으로삼  다른분들은놋북어디서사셨나요  인터넷  주문해서  난  애플 온라인에서  뭐사야될지모르겠다  학생할인받음  호옹  총학에서 할인하는거있어서  맥북삼  그걸로  게다 내 맥북은 터스텀맥북이라  주문제작해서 공장에서 바로나온거임  주문제작    누나가사양선택한거    당연  한걸  램 올리고  끝낫당  씨퓨올림    난아는게없으니걍사야겠다  그냥  가벼운게  장땡    난 주코딩은  뎃탑에서함  그냥 수업필기나하지  놋북으로요즘은  뎃탑아니면 안돌아감 요즘은    고사양임  요즘하는일이  나름대로  나  인터넷으로  주문햇어  다나와  에서  비교해서삿어  네이버에서검색햇엇음  다나와  다나와에서  가격비교가능  대충 견적낼수있음  운영체제  미포함으로  삿어  운영체제 근데 미포함으로 하면  얼마임  내가 깔라고  이면사나  한   정도  아하  싸지는거  같던데  난 정품하나있어서  그거씀  

In [106]:
word_list = [k for (k,v) in kkma.pos(text_edit) if v.startswith("NN")]

In [124]:
for i in word_list:
    if len(i) == 1:
        word_list.remove(i)
word_list

['민방위',
 '교육',
 '민방위',
 '라니',
 '개심',
 '기간',
 '다음주',
 '월요일',
 '금요일',
 '대구',
 '잉여',
 '코딩',
 '직장인',
 '시간',
 '언어',
 '공부',
 '수임',
 '감기',
 '복학',
 '복학',
 '학기',
 '작년',
 '여름',
 '그때',
 '유럽',
 '오늘',
 '민방위',
 '신세',
 '한탄',
 '안보',
 '교육',
 '거함',
 '민방위',
 '하면',
 '시간',
 '주가',
 '간고',
 '사임',
 '휴가',
 '유나',
 '누나',
 '인터넷',
 '인터넷',
 '주문',
 '애플',
 '온라인',
 '학생',
 '할인',
 '할인',
 '북삼',
 '게다',
 '주문',
 '제작',
 '공장',
 '주문',
 '제작',
 '누나',
 '사양',
 '선택',
 '올리고',
 '올림',
 '사야',
 '장땡',
 '코딩',
 '수업',
 '필기',
 '요즘',
 '요즘',
 '고사',
 '요즘',
 '나름',
 '인터넷',
 '주문',
 '다나',
 '비교',
 '네이버',
 '검색',
 '다나',
 '다나',
 '가격',
 '비교',
 '가능',
 '대충',
 '견적',
 '있음',
 '운영',
 '체제',
 '포함',
 '운영',
 '체제',
 '포함',
 '얼마',
 '내가',
 '이면사',
 '정도',
 '정품',
 '파일',
 '구함',
 '아악',
 '무게',
 '중요',
 '가보',
 '그램',
 '범인',
 '정상',
 '그램',
 '발열',
 '견적',
 '얼마',
 '그램',
 '그램',
 '그램',
 '그램',
 '개가',
 '안됨',
 '그램',
 '선전',
 '프로',
 '에어',
 '인가',
 '그램',
 '케이스',
 '삼성',
 '발열',
 '하대',
 '윈도',
 '내고',
 '오피스',
 '삼성',
 '시리즈',
 '학생',
 '공짜',
 '요즘',
 '유료',
 '학생',
 '공짜',
 '광운',
 '대도',
 '유료',
 '잘만',
 '

In [131]:
word_count = Counter(word_list)

In [132]:
word_count = word_count.most_common(30)

In [133]:
tag_list = pytagcloud.make_tags(word_count, maxsize=100, minsize=20)

In [134]:
pytagcloud.create_tag_image(tag_list, "cloud_kakaotalk.jpg", fontname="NanumGothic")

한 문서에서의 단어의 빈도수 Term Frequency<br>
그리고 해당 단어가 다른 문서에서 얼마나 나왔는지 Document Frequency<br>
를 결합한 방식이<br>
TF-IDF 방식

In [135]:
import requests
from bs4 import BeautifulSoup

In [136]:
headers = {
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
}

params = {
}

url = "https://media.daum.net/"
resp = requests.get(url, params=params, headers=headers) #해당 주소에 대해서 parameter와 header를 넣어서 request
dom = BeautifulSoup(resp.text, "lxml") # BeautifulSoup을 이용해서 텍스트로 받아온다.
aList = dom.select(".list_headline .tit_g > a") #해당하는 태그 내용을 갖고 온다 (주소를 갖고온다)
links = []
for row in aList:
    links.append(row["href"]) # 주소들을 리스트에 넣어준다.

# daum 뉴스에서의 링크들을 뽑아왔다.

In [137]:
links

['http://v.media.daum.net/v/20181028164903850',
 'http://v.media.daum.net/v/20181028164604773',
 'http://v.media.daum.net/v/20181028164104633',
 'http://v.media.daum.net/v/20181028164102632',
 'http://v.media.daum.net/v/20181028163851574',
 'http://v.media.daum.net/v/20181028163257414',
 'http://v.media.daum.net/v/20181028163253411',
 'http://v.media.daum.net/v/20181028163145387',
 'http://v.media.daum.net/v/20181028163000336',
 'http://v.media.daum.net/v/20181028162436248',
 'http://v.media.daum.net/v/20181028161236972',
 'http://v.media.daum.net/v/20181028161200956']

In [139]:
for link in links:
    docid = link[-17:]
    resp = requests.get(link, headers=headers)
    html = BeautifulSoup(resp.text, "lxml")
    content = html.select(".article_view")
    
    with open("D:/Study/BigData/JupyterNotebook/Study/doc/{0}".format(docid), "w", encoding='utf-8') as fp:
        fp.write(content[0].text)
        
# 기사 내용을 갖고와서 로컬에 파일로 저장한다.

In [140]:
import glob # 경로이름을 찾아주는 모듈
import re # 정규식을 쓸 때 사용하는 모듈

In [141]:
fileList = glob.glob("D:/Study/BigData/JupyterNotebook/Study/doc/*")
# 해당 path안에 있는 파일의 path를 찾아준다.

In [142]:
with open(fileList[0], "r", encoding='utf-8') as fp: 
    content = fp.read() # 파일을 읽어들임.
    content = re.sub(r"[\s]{2,}", "", content) #정규식을 이용해서 공백이 2개 이상 붙은 것들을 없애준다.

In [143]:
import nltk

In [156]:
for sentence in nltk.sent_tokenize(content):
#     words = nltk.word_tokenize(sentence) # nltk로 단어를 tokenize해준다.
    words = nltk.regexp_tokenize(sentence, r"[0-9ㄱ-ㅎㅏ-ㅣ가-힣]+") # 한글과 숫자에 대해서만 허용한 정규식에 따라 영어를 빼고 tokenize해준다.
#     sentence = re.sub( r"[\s\W]+", "", sentence) # 이 경우 WhiteSpace를 다 없애준다.
#     words = []
    
#     for i in range(len(sentence)):
#         words.append(sentence[i:i+2]) #근처의 2개의 글자를 합쳐서 bigram을 만들어준다.
    for word in words:
        tokens = []
        for i in range(len(word)):
            if i+1 < len(word) :
                tokens.append(word[i:i+2])
        print(tokens)
#         print(word)
#         print(tokens, end = "\n\n")
#     print(sentence)
#     print(words, end = "\n\n") # 토큰화된 단어 리스트를 출력해준다.

# 한 글자씩 인접한 1개 글자와 쌍을 맞춰서 list를 생성했다.

['바른', '른미', '미래', '래당']
['등이']
['고발', '발한']
['사건', '건의']
['피고', '고발', '발인']
['신분', '분으', '으로']
['경찰', '찰조', '조사', '사를']
['하루']
['앞둔']
['이재', '재명']
['경기', '기도', '도지', '지사', '사가']
['유명']
['로펌', '펌인']
['법무', '무법', '법인']
['화우']
['변호', '호사', '사를']
['선임', '임하', '하고']
['법리']
['검토']
[]
['조사']
['준비', '비를']
['마친']
['것으', '으로']
['확인', '인됐', '됐다']
[]
['지사', '사는']
['경찰', '찰조', '조사', '사가']
['진행', '행되', '되는']
['29', '9일']
['하루']
['휴가', '가를']
[]
['상황', '황으', '으로']
['이날']
['경기', '기도', '도청', '청으', '으로']
['출근', '근하', '하지']
['않고']
['자택', '택에', '에서']
['곧바', '바로']
['경기']
['분당', '당경', '경찰', '찰서', '서에']
[]
['예정', '정이', '이다']
['이재', '재명']
['경기', '기도', '도지', '지사']
['사진']
['이한', '한형']
['기자']
['이지', '지사']
[]
['법리', '리검', '검토']
[]
['모든']
['조사', '사준', '준비']
['마쳐']
['문제']
['없을']
[]
[]
['지사', '사가']
['변호', '호인', '인으', '으로']
['선임', '임한']
['화우']
[]
['20', '00', '03', '3년']
['설립', '립돼']
['20', '00', '07']
['20', '01', '11', '1년']
['퇴직']
['검사']
['16', '66', '6명']
['가운', '운데']
['대형', '형로', '로펌', '펌에']
['취직', '직한']
['46', 

In [153]:
from konlpy.tag import Kkma

In [157]:
uniqueNouns = []


for sentence in nltk.sent_tokenize(content):
    nouns = []
    for word in nltk.word_tokenize(sentence):
        nouns.extend(Kkma().nouns(word)) # 검색 엔진에서도 명사로서 간단히 검색할 때 검색 성능이 제일 좋다.
        nouns = list(set(nouns)) #unique한 단어들만 가져온다.
    uniqueNouns.extend(nouns)
    uniqueNouns = list(set(uniqueNouns)) # set을 통해서 중복되는 것들 없애준다.
#     print(sentence)
#     print(nouns)
#     print(sentence)
#     print(words)
print(len(uniqueNouns))

# 명사들만 가져오는데 중복을 허용하지 않도록 했다.

294
